In [ ]:
import os
import warnings

warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
plt.rcParams["figure.figsize"] = (8, 6)
plt.rcParams["figure.dpi"] = 100
plt.style.use("bmh")

In [ ]:
os.chdir("../data/")

# Importowanie danych i ich przygotowanie

In [ ]:
RANDOM_STATE = 0
TEST_SIZE = 0.2

In [ ]:
df = pd.read_csv("totrain.csv", index_col=0)
df = df.drop("dl_polaczenia", axis=1)

In [ ]:
X, y = df.drop("target", axis=1), df.target

#### train-test-validate split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
)
X_test, X_validate, y_test, y_validate = train_test_split(
    X_test, y_test, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

#### DO POPRAWY

In [ ]:
serie = y_train.value_counts()
ratio = {1: serie[0] / serie[1]}
serie

In [ ]:
df.head()

## Przygotowanie modeli

In [ ]:
def build_model(model, name):
    global X_train, y_train, X_test, y_test, decomp
    plt.figure(dpi=50)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print("-" * 3 + " " + name + " " + "-" * 3)
    print(classification_report(y_test, y_pred))
    print("-" * 10)
    print("Area under curve\n", roc_auc_score(y_test, y_pred))

    fpr, tpr, *_ = roc_curve(y_test, y_pred)
    plt.plot(fpr, tpr)

    print("-" * 10)
    print("Crosstab")
    print(pd.crosstab(y_test, y_pred, rownames=["Actual"], colnames=["Predicted"]))

In [ ]:
lr = LogisticRegression(max_iter=10000, class_weight=ratio)
svm = LinearSVC(C=0.0001, class_weight=ratio)
dtc = DecisionTreeClassifier(
    max_depth=10,
    random_state=RANDOM_STATE,
    class_weight=ratio,
)
rfc = RandomForestClassifier(
    n_estimators=10000,
    n_jobs=8,
    random_state=RANDOM_STATE,
    class_weight=ratio,
)
knn = KNeighborsClassifier(n_neighbors=49)

models = [
    (lr, "LogisticRegression"),
    (svm, "SVM"),
    (dtc, "DecisionTree"),
    (rfc, "RandomForest"),
]

### Ewaluacja modeli

In [ ]:
build_model(knn, "KNN")

In [ ]:
build_model(lr, "LogisticRegression")

In [ ]:
build_model(svm, "SupportedVectorMachines")

In [ ]:
build_model(dtc, "DecisionTreeClassifier")

In [ ]:
build_model(rfc, "RandomForestClassifier")

#### Na podstawie powyższych wyników, możemy zaobserwować, że najlepszym wyborem będzie algorytm _Decision tree_.

In [ ]:
n_components = list(range(1, X.shape[1] + 1, 1))
criterion = ["gini", "entropy"]
max_depth = list(range(2, 51, 2))
parameters = dict(
    criterion=criterion,
    max_depth=max_depth,
    class_weight=[ratio],
    max_features=["sqrt", "auto"],
)

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(), parameters, scoring="f1")

#### Walidacja na podstawie wcześniejszego "odłożenia" podzbioru z suffixem _validate_

In [ ]:
gs.fit(X_validate, y_validate)

In [ ]:
gs.best_params_

In [ ]:
dct = DecisionTreeClassifier(
    max_features="sqrt", max_depth=8, criterion="entropy", class_weight=ratio
)
build_model(dct, "DCT")

In [ ]:
#### Ważność konkretnych cech dla modelu

In [ ]:
pd.Series(dct.feature_importances_, index=X_validate.columns).nlargest(10).plot(
    kind="barh"
)

# Wnioski

Podstawowe algorytmy do binarnej klasyfikacji, okazały się być niewystarczające. 

*DOPISZ*


## Co możnaby zrobić inaczej?

Przede wszystkim, należałoby jeszcze raz przeprowadzić proces czyszczenia danych i proces inżynierii cech. Jest parę działań, które popełniliśmy jednak nie byliśmy ich pewni.

*DOPISZ*

- Nie usuwaliśmy żadnego rekordu, nawet dla połączeń trwających 0 sekund, bo założyliśmy, że takie połączenia mogą się zdarzyć.
- Nie usuwaliśmy rekordów, gdzie połączenie trwało powyżej 3000 sekund, bo chcieliśmy przygotować model pod obsługę długich rozmów.
- Można przyjąć _unknown_ jako klasę i nie przybliżać wartości cech, gdzie dana obserwacja, zawiera cechę o wartości _unknown_.
- Należałoby się zastanowić nad cechą _liczba\_dni\_od\_ost\_kontaktu_, gdzie w naszym procesie, klasyfikowaliśmy wartości do zadanych przedziałów, co być może nie jest odpowiednie, bo dla osób, z którymi wcześniej się nie kontaktowano, zaklasyfikowalismy ich do ostatniego przedziału. Takie rozwiązanie, 

## Key-takeaways, na co należy zwrócić uwagę przy następnych kampaniach

*DOPISZ*

Ladnie napisac, trzy najważniejsze cechy na podstawie wykresu _feature\_importances_ i opisać jakieś propozycje.